# Examples of running IHPF 

1. Download the scRNA-seq datasets in h5ad format and put in the folder Data/ 

Creating a smaller sample of 10Xmouse, does not work as the sub-sampled cells does not represent the distribution of original samples, making these cells not usable for IHPF. 


In [58]:
import IHPF
import scanpy as sc
from anndata import AnnData

In [48]:
from scipy.sparse import coo_matrix
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd

In [69]:
datasets = ['10Xmouse']
for dataset in datasets:
    ## Replace this with the location of your h5ad files
    adata = sc.read('../Data/{}.h5ad'.format(dataset))
    print(adata)
    selected_index = [x for x in range(1000,1200)] + [x for x in range(4000,4200)] + [x for x in range(6000,6200)]
    newX = adata.X[selected_index,:]
    newobs = adata.obs.iloc[selected_index,:]
    newadata = AnnData(X=newX,obs=newobs)
    newadata.write('../Data/{}_sample.h5ad'.format(dataset))

AnnData object with n_obs × n_vars = 9530 × 13130
    obs: 'actual', 'batch', 'IHPF_kmeans_normalised', 'HPF_kmeans_normalised', 'INMF_kmeans_normalised', 'IPCA_kmeans_normalised', 'INMF_2_kmeans_normalised'
    obsm: 'HPF', 'IHPF', 'INMF', 'INMF_2', 'IPCA'
    varm: 'HPF', 'IHPF', 'INMF', 'INMF_2'


In [70]:
adata = sc.read('../Data/{}_sample.h5ad'.format(dataset))

In [75]:
datasets = ['10Xmouse']
for dataset in datasets:
    ## Replace this with the location of your h5ad files
    adata = sc.read('../Data/{}_sample.h5ad'.format(dataset))
    adataobs = adata.obs.copy()
    no_cell_types = len(adataobs['actual'].unique())
    adataobs.reset_index(inplace=True)
    Xlist = list()
    for i,df in adataobs.groupby('batch'):
        batchidx = df.index
        Xlist.append(coo_matrix(adata.X[batchidx,:]))
    model = IHPF.scIHPF(no_cell_types,max_iter=500)
    model.fit(Xlist)
    adata.obsm['IHPF_{}'.format(no_cell_types)] = np.concatenate(model.cell_scores(),axis=0)
    adata.varm['IHPF_{}'.format(no_cell_types)] = model.shared_gene_scores()
    kmeans_cell = KMeans(n_clusters=no_cell_types, random_state=0).fit(normalize(adata.obsm['IHPF_{}'.format(no_cell_types)]))
    adata.obs['IHPF_{}_kmeans_normalised'.format(no_cell_types)] = kmeans_cell.labels_
    batch = dict()
    actual = dict()
    for method in ['IHPF_2','IHPF','HPF','INMF','IPCA']:
        batch[method] = adjusted_mutual_info_score(adata.obs['batch'],adata.obs['{}_kmeans_normalised'.format(method)])
        actual[method] = adjusted_mutual_info_score(adata.obs['actual'],adata.obs['{}_kmeans_normalised'.format(method)])
    batch['actual'] = adjusted_mutual_info_score(adata.obs['batch'],adata.obs['actual'])
    actual['batch'] = adjusted_mutual_info_score(adata.obs['batch'],adata.obs['actual'])
    batchAMI[dataset] = batch
    actualAMI[dataset] = actual

Clipping dp: was [0.00011484677088446915, 7.743774767732248e-05, 0.00047286757035180926] now [4.949536523781717e-07, 3.996510640718043e-07, 4.5454432256519793e-07]
[Iter.    0]  loss:114.020537  pct:100.000000000
[Iter.   10]  loss:8.444018  pct:-92.594300649
[Iter.   20]  loss:7.514529  pct:-11.007665951
[Iter.   30]  loss:6.937852  pct:-7.674160907
[Iter.   40]  loss:6.829657  pct:-1.559479498
[Iter.   50]  loss:6.813843  pct:-0.231552662
[Iter.   60]  loss:6.819152  pct:0.077912924
[Iter.   70]  loss:6.832743  pct:0.199298055
[Iter.   80]  loss:6.843990  pct:0.164604784
getting worse break


In [76]:
pd.DataFrame(actualAMI)

,humanpancreas,10Xmouse
HPF_10,0.133217,NaN
IHPF,0.276381,1.000000
HPF,0.141297,0.001738
INMF,0.238332,0.058434
IPCA,0.211085,0.750244
batch,0.031719,0.756558
IHPF_2,NaN,0.766756


## IHPF 

In [49]:
datasets = ['humanpancreas','10Xmouse','10Xpbmc','mixedpbmc']
for dataset in datasets:
    ## Replace this with the location of your h5ad files
    adata = sc.read('../Data/{}.h5ad'.format(dataset))
    adataobs = adata.obs.copy()
    no_cell_types = len(adataobs['actual'].unique())
    adataobs.reset_index(inplace=True)
    Xlist = list()
    for i,df in adataobs.groupby('batch'):
        batchidx = df.index
        Xlist.append(coo_matrix(adata.X[batchidx,:]))
    model = IHPF.scIHPF(no_cell_types,max_iter=500)
    model.fit(Xlist)
    adata.obsm['IHPF_{}'.format(no_cell_types)] = np.concatenate(model.cell_scores(),axis=0)
    adata.varm['IHPF_{}'.format(no_cell_types)] = model.shared_gene_scores()
    kmeans_cell = KMeans(n_clusters=no_cell_types, random_state=0).fit(normalize(adata.obsm['IHPF_{}'.format(no_cell_types)]))
    adata.obs['IHPF_{}_kmeans_normalised'.format(no_cell_types)] = kmeans_cell.labels_
    print(adata)
    

Clipping dp: was [2.054972355836071e-06, 2.5971088689402677e-05, 6.098692392697558e-05, 3.414987759242649e-07, 4.0265252465587764e-08] now [1.030316692776978e-06, 2.0419471547938884e-07, 4.648955073207617e-07, 2.013995617744513e-08, 4.222342795401346e-09]
[Iter.    0]  loss:3729.622879  pct:100.000000000
[Iter.   10]  loss:166.439517  pct:-95.537363352
[Iter.   20]  loss:139.270597  pct:-16.323599914
[Iter.   30]  loss:132.457036  pct:-4.892318137
[Iter.   40]  loss:130.263620  pct:-1.655945074
[Iter.   50]  loss:129.058297  pct:-0.925295289
[Iter.   60]  loss:128.399832  pct:-0.510207721
[Iter.   70]  loss:127.999939  pct:-0.311442905
[Iter.   80]  loss:127.715797  pct:-0.221986702
[Iter.   90]  loss:127.523018  pct:-0.150943254
[Iter.  100]  loss:127.389771  pct:-0.104488408
[Iter.  110]  loss:127.279905  pct:-0.086244184
[Iter.  120]  loss:127.194298  pct:-0.067259001
[Iter.  130]  loss:127.109281  pct:-0.066839931
[Iter.  140]  loss:127.034315  pct:-0.058977855
[Iter.  150]  loss:1

Clipping dp: was [6.3929469433787744e-06, 4.0064984204946086e-05, 4.836061179958051e-06] now [6.771162152290344e-06, 3.1818742863833905e-06, 3.004571422934532e-06]
[Iter.    0]  loss:54.305355  pct:100.000000000
[Iter.   10]  loss:8.905304  pct:-83.601426611
[Iter.   20]  loss:8.278822  pct:-7.034930887
[Iter.   30]  loss:8.028088  pct:-3.028620566
[Iter.   40]  loss:7.944158  pct:-1.045443677
[Iter.   50]  loss:7.914899  pct:-0.368319620
[Iter.   60]  loss:7.897272  pct:-0.222701106
[Iter.   70]  loss:7.884289  pct:-0.164393892
[Iter.   80]  loss:7.873247  pct:-0.140052769
[Iter.   90]  loss:7.864083  pct:-0.116390326
[Iter.  100]  loss:7.857261  pct:-0.086758563
[Iter.  110]  loss:7.851978  pct:-0.067234398
[Iter.  120]  loss:7.847421  pct:-0.058033702
[Iter.  130]  loss:7.843998  pct:-0.043621968
[Iter.  140]  loss:7.841144  pct:-0.036376927
[Iter.  150]  loss:7.838894  pct:-0.028701340
[Iter.  160]  loss:7.837117  pct:-0.022670605
[Iter.  170]  loss:7.835657  pct:-0.018624191
[Iter

In [31]:
adata.obs

,actual,batch,IHPF_kmeans_normalised,HPF_kmeans_normalised,INMF_kmeans_normalised,IPCA_kmeans_normalised,IHPF_2_kmeans_normalised
0,293t,0,1,0,1,0,0
1,293t,0,1,0,1,0,0
2,293t,0,1,0,1,0,0
3,293t,0,1,0,1,0,0
4,293t,0,1,0,1,0,0
...,...,...,...,...,...,...,...
9525,jurkat,2,0,1,0,0,1
9526,293t,2,1,1,1,0,0
9527,293t,2,1,1,1,0,0
9528,293t,2,1,1,1,0,0


In [32]:
from sklearn.metrics import adjusted_mutual_info_score, silhouette_score
datasets = ['humanpancreas','10Xmouse','10Xpbmc','mixedpbmc']
batchAMI = dict()
actualAMI = dict()

In [33]:
for dataset in datasets:
    batch = dict()
    actual = dict()
    adata = sc.read('../Data/{}.h5ad'.format(dataset))
    for method in ['IHPF_2','IHPF','HPF','INMF','IPCA']:
        batch[method] = adjusted_mutual_info_score(adata.obs['batch'],adata.obs['{}_kmeans_normalised'.format(method)])
        actual[method] = adjusted_mutual_info_score(adata.obs['actual'],adata.obs['{}_kmeans_normalised'.format(method)])
    batch['actual'] = adjusted_mutual_info_score(adata.obs['batch'],adata.obs['actual'])
    actual['batch'] = adjusted_mutual_info_score(adata.obs['batch'],adata.obs['actual'])
    batchAMI[dataset] = batch
    actualAMI[dataset] = actual

In [34]:
pd.DataFrame(batchAMI)

,humanpancreas,10Xmouse,newpbmc,hpfpbmc
IHPF_2,0.498353,0.498353,0.498353,0.498353
IHPF,0.498353,0.498353,0.498353,0.498353
HPF,0.744954,0.744954,0.744954,0.744954
INMF,0.288737,0.288737,0.288737,0.288737
IPCA,0.738750,0.738750,0.738750,0.738750
actual,0.498373,0.498373,0.498373,0.498373


In [35]:
pd.DataFrame(actualAMI)

,humanpancreas,10Xmouse,newpbmc,hpfpbmc
IHPF_2,0.976076,0.976076,0.976076,0.976076
IHPF,0.976076,0.976076,0.976076,0.976076
HPF,-0.000067,-0.000067,-0.000067,-0.000067
INMF,0.226908,0.226908,0.226908,0.226908
IPCA,0.447701,0.447701,0.447701,0.447701
batch,0.498373,0.498373,0.498373,0.498373


## INMF 

In [26]:
from INMF import INMF

In [27]:
class scINMF:
    def __init__(self, k, alpha=1, **kwargs):
        np.random.seed(0)
        self.n_components = k
        self.method = INMF(
            n_components=self.n_components, solver="mu", alpha=alpha, **kwargs
        )

    def fit(self, X):
        self.data = X
        (
            self.cell_scores,
            self.shared_gene_scores,
            self.dataset_gene_scores,
        ) = self.method.fit_transform(self.data)

    def count_matrix(self):
        # print(self.cell_score[0].shape)
        # print(self.dataset_gene_score[0].shape)
        # print(self.shared_gene_score.shape)
        original = [
            np.dot(
                self.cell_score[i], self.shared_gene_score + self.dataset_gene_score[i]
            )
            for i in range(len(self.cell_score))
        ]
        return original

    def explained_deviance(self, X, X_rep, beta):
        try:
            X_avg = coo_matrix(
                np.matmul(np.ones((X.shape[0], 1)), X.mean(axis=0).reshape(1, -1))
            )
            average_divergence = beta_divergence_ppc(X, X_avg, beta)
            model_divergence = beta_divergence_ppc(X, X_rep, beta)
            ratio = (average_divergence - model_divergence) / average_divergence
            return ratio
        except:
            print("Error in calculating deviance")
            return 0

In [32]:
datasets = ['humanpancreas']
for dataset in datasets:
    ## Replace this with the location of your h5ad files
    adata = sc.read('../Data/{}.h5ad'.format(dataset))
    adataobs = adata.obs.copy()
    no_cell_types = len(adataobs['actual'].unique())
    adataobs.reset_index(inplace=True)
    Xlist = list()
    for i,df in adataobs.groupby('batch'):
        batchidx = df.index
        Xlist.append(coo_matrix(adata.X[batchidx,:]))
    model = scINMF(no_cell_types,max_iter=500)
    model.fit(Xlist)
    adata.obsm['INMF_{}'.format(no_cell_types)] = np.concatenate(model.cell_scores,axis=0)
    adata.varm['INMF_{}'.format(no_cell_types)] = model.shared_gene_scores.transpose()
    kmeans_cell = KMeans(n_clusters=no_cell_types, random_state=0).fit(normalize(adata.obsm['INMF_{}'.format(no_cell_types)]))
    adata.obs['INMF_{}_kmeans_normalised'.format(no_cell_types)] = kmeans_cell.labels_
    adata.write('../Data/{}.h5ad'.format(dataset))

Reconstruction error 1120212.068773359


In [33]:
from sklearn.metrics import adjusted_mutual_info_score, silhouette_score
datasets = ['humanpancreas']
batchAMI = dict()
actualAMI = dict()

for dataset in datasets:
    batch = dict()
    actual = dict()
    adata = sc.read('../Data/{}.h5ad'.format(dataset))
    for method in ['INMF_10','IHPF','HPF','INMF','IPCA']:
        batch[method] = adjusted_mutual_info_score(adata.obs['batch'],adata.obs['{}_kmeans_normalised'.format(method)])
        actual[method] = adjusted_mutual_info_score(adata.obs['actual'],adata.obs['{}_kmeans_normalised'.format(method)])
    batch['actual'] = adjusted_mutual_info_score(adata.obs['batch'],adata.obs['actual'])
    actual['batch'] = adjusted_mutual_info_score(adata.obs['batch'],adata.obs['actual'])
    batchAMI[dataset] = batch
    actualAMI[dataset] = actual

In [34]:
pd.DataFrame(actualAMI)

,humanpancreas
HPF,0.141297
IHPF,0.276381
INMF,0.238332
INMF_10,0.199625
IPCA,0.211085
batch,0.031719


## scHPF

In [38]:
from schpf import scHPF
from scipy.sparse import vstack

In [42]:
datasets = ['humanpancreas']
for dataset in datasets:
    ## Replace this with the location of your h5ad files
    adata = sc.read('../Data/{}.h5ad'.format(dataset))
    adataobs = adata.obs.copy()
    no_cell_types = len(adataobs['actual'].unique())
    adataobs.reset_index(inplace=True)
    Xlist = list()
    for i,df in adataobs.groupby('batch'):
        batchidx = df.index
        Xlist.append(coo_matrix(adata.X[batchidx,:]))
    model = scHPF(no_cell_types,max_iter=500)
    model.fit(vstack(Xlist))
    adata.obsm['HPF_{}'.format(no_cell_types)] = model.cell_score()
    adata.varm['HPF_{}'.format(no_cell_types)] = model.gene_score()
    kmeans_cell = KMeans(n_clusters=no_cell_types, random_state=0).fit(normalize(adata.obsm['HPF_{}'.format(no_cell_types)]))
    adata.obs['HPF_{}_kmeans_normalised'.format(no_cell_types)] = kmeans_cell.labels_
    adata.write('../Data/{}.h5ad'.format(dataset))

[Iter.    0]  loss:707.795400  pct:100.000000000
[Iter.   10]  loss:27.256316  pct:-96.149124950
[Iter.   20]  loss:21.592846  pct:-20.778562361
[Iter.   30]  loss:20.945739  pct:-2.996855619
[Iter.   40]  loss:20.653689  pct:-1.394317453
[Iter.   50]  loss:20.561153  pct:-0.448038017
[Iter.   60]  loss:20.528952  pct:-0.156609043
[Iter.   70]  loss:20.508578  pct:-0.099245690
[Iter.   80]  loss:20.492615  pct:-0.077837435
[Iter.   90]  loss:20.479040  pct:-0.066243636
[Iter.  100]  loss:20.469607  pct:-0.046061907
[Iter.  110]  loss:20.460487  pct:-0.044551732
[Iter.  120]  loss:20.453325  pct:-0.035004752
[Iter.  130]  loss:20.448780  pct:-0.022220218
[Iter.  140]  loss:20.445213  pct:-0.017443994
[Iter.  150]  loss:20.442168  pct:-0.014894396
[Iter.  160]  loss:20.439343  pct:-0.013820324
[Iter.  170]  loss:20.436525  pct:-0.013784295
[Iter.  180]  loss:20.434212  pct:-0.011318608
[Iter.  190]  loss:20.431732  pct:-0.012139555
[Iter.  200]  loss:20.428680  pct:-0.014936018
[Iter.  2

In [45]:
from sklearn.metrics import adjusted_mutual_info_score, silhouette_score
datasets = ['humanpancreas']
batchAMI = dict()
actualAMI = dict()

for dataset in datasets:
    batch = dict()
    actual = dict()
    adata = sc.read('../Data/{}.h5ad'.format(dataset))
    for method in ['HPF_10','IHPF','HPF','INMF','IPCA']:
        batch[method] = adjusted_mutual_info_score(adata.obs['batch'],adata.obs['{}_kmeans_normalised'.format(method)])
        actual[method] = adjusted_mutual_info_score(adata.obs['actual'],adata.obs['{}_kmeans_normalised'.format(method)])
    batch['actual'] = adjusted_mutual_info_score(adata.obs['batch'],adata.obs['actual'])
    actual['batch'] = adjusted_mutual_info_score(adata.obs['batch'],adata.obs['actual'])
    batchAMI[dataset] = batch
    actualAMI[dataset] = actual

In [46]:
pd.DataFrame(actualAMI)

,humanpancreas
HPF,0.141297
HPF_10,0.133217
IHPF,0.276381
INMF,0.238332
IPCA,0.211085
batch,0.031719
